# Prerequisites

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import zipfile
import tempfile
from scipy.ndimage import laplace
from scipy.stats import skew, kurtosis, entropy
from skimage.metrics import structural_similarity as ssim
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import cv2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
import random
from torchvision import transforms
import torchvision.transforms.functional as TF
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import spearmanr, kendalltau
import torchvision.transforms as T
import random
import torchvision.models as models

# Data

## Data Loading

In [ ]:
PATH = "/kaggle/input/image-noise/Archive"
TRAIN_PATH = os.path.join(PATH, 'train.csv')
VAL_PATH = os.path.join(PATH, 'validation.csv')
TEST_PATH = os.path.join(PATH, 'test.csv')
SAMPLES_PATH = os.path.join(PATH, 'samples')
def load_samples():
    samples = []
    print(f"Loading samples and extracting features from: {SAMPLES_PATH}")

    for filename in tqdm(os.listdir(SAMPLES_PATH)):
        if filename.endswith('.npy'):
            image_id = os.path.splitext(filename)[0]
            image_path_full = os.path.join(SAMPLES_PATH, filename)

            try:
                image = np.load(image_path_full)
                image = image.astype(np.float32)
                samples.append({"sample":image, "id":image_id})
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    df_samples = pd.DataFrame(samples)
    df_samples.set_index('id', inplace=True)
    return df_samples

def load():
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Shape of df_train: {df_train.shape}")
    df_val = pd.read_csv(VAL_PATH)
    print(f"Shape of df_val: {df_val.shape}")
    df_test = pd.read_csv(TEST_PATH)
    print(f"Shape of df_test: {df_test.shape}")
    df_samples = load_samples()
    print(f"\nShape of df_samples: {df_samples.shape}")
    return {
        "samples": df_samples,
        "train": df_train,
        "val": df_val,
        "test": df_test
    }

data = load()

## General Dataset Analysis

### Pie Chart of Dataset Distribution

In [ ]:
def display_pie_chart(data, filename):
    values = [data['train'].shape[0], data['val'].shape[0], data['test'].shape[0]]
    labels = ['Train', 'Validation', 'Test']
    
    def make_autopct(all_values):
        def my_autopct(pct):
            total = sum(all_values)
            val = int(round(pct * total / 100.0))
            return '{p:.1f}%\n({v:d})'.format(p=pct, v=val)
        return my_autopct

    plt.figure(figsize=(8, 6))
    
    plt.pie(
        values, 
        labels=labels, 
        autopct=make_autopct(values),
    )
    
    plt.title(filename)
    plt.axis('equal') 

    plt.savefig(filename)
    plt.show()

display_pie_chart(data, filename="example_distribution.svg")

### Label Distribution Pie Charts

In [ ]:
def display_label_distribution(df, filename):
    label_counts = df['label'].value_counts().sort_index()

    plt.figure(figsize=(4, 4))

    plt.pie(
        label_counts.values, 
        labels=label_counts.index, 
        autopct='%1.1f%%',  # Show percentages with 1 decimal place
        startangle=90       # Rotate start to vertical
    )
    
    plt.title(filename)
    
    plt.savefig(filename)
    plt.show()

# Usage
display_label_distribution(data['train'], filename="train_label_dist.svg")
display_label_distribution(data['val'], filename="val_label_dist.svg")

### Global Statistics

In [ ]:
def analyze_and_plot_histograms(df,filename):
    pixel_sum = 0.0
    pixel_sum_sq = 0.0
    pixel_count = 0
    global_min = np.inf
    global_max = -np.inf

    image_stats = {
        'means': [],
        'stds': [],
        'mins': [],
        'maxs': []
    }

    print(f"Processing {len(df)} images...")

    for img in tqdm(df['sample']):
        flat = img.flatten().astype(np.float64)
        
        current_min = np.min(flat)
        current_max = np.max(flat)
        if current_min < global_min: global_min = current_min
        if current_max > global_max: global_max = current_max
        
        pixel_sum += np.sum(flat)
        pixel_sum_sq += np.sum(flat ** 2)
        pixel_count += len(flat)

        image_stats['means'].append(np.mean(flat))
        image_stats['stds'].append(np.std(flat))
        image_stats['mins'].append(current_min)
        image_stats['maxs'].append(current_max)

    total_mean = pixel_sum / pixel_count
    # Variance = E[x^2] - (E[x])^2
    total_var = (pixel_sum_sq / pixel_count) - (total_mean ** 2)
    total_std = np.sqrt(total_var)

    print(f"Global Stats -> Mean: {total_mean:.2f}, Std: {total_std:.2f}, Min: {global_min}, Max: {global_max}")

    fig = plt.figure(figsize=(16, 12))
    
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
    subplots = [
        (1, 0, 'means', 'Distribution of Image Means'),
        (1, 1, 'stds',  'Distribution of Image Contrast (Std)'),
        (0, 0, 'mins',  'Distribution of Image Mins'),
        (0, 1, 'maxs',  'Distribution of Image Maxs')
    ]
    
    for row, col, key, title in subplots:
        ax = plt.subplot2grid((2, 2), (row, col))
        sns.histplot(image_stats[key], kde=True, ax=ax, color='steelblue')
        ax.set_title(title)
        ax.set_xlabel(key.capitalize())

    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

# Run it
analyze_and_plot_histograms(data['samples'], 'global-stats.svg')

## Sample Analysis

In [ ]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def calculate_glcm_correlation_custom_range(image):
    min_val, max_val = -59, 56
    num_levels = 1120

    img_clipped = np.clip(image, min_val, max_val)
    img_quantized = ((img_clipped - min_val) / (max_val - min_val) * (num_levels - 1)).astype(np.uint8)

    glcm = graycomatrix(
        img_quantized, 
        distances=[1], 
        angles=[0], 
        levels=num_levels, 
        symmetric=True, 
        normed=True
    )

    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    return correlation

from skimage.feature import graycomatrix, graycoprops

def get_glcm_contrast(image):
    img_norm = (image + 59).astype(np.uint8)
    glcm = graycomatrix(img_norm, [1], [0], levels=120, symmetric=True, normed=True)

    return graycoprops(glcm, 'contrast')[0, 0]

import numpy as np
from skimage.feature import local_binary_pattern

def get_lbp_smoothness_bin(image, radius=1, n_points=8):
    img_norm = (image + 59).astype(np.uint8)

    lbp = local_binary_pattern(img_norm, n_points, radius, method='default')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(2**n_points + 1), density=True)
    return hist[0]

def get_lbp_uniform_regularity(image, radius=1, n_points=8):
    img_norm = (image + 50).astype(np.uint8)
    lbp = local_binary_pattern(img_norm, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(n_points + 3), density=True)
    return np.sum(hist[:-1])

def get_glcm_homogeneity(image):
    img_norm = (image + 59).astype(np.uint8)
    
    glcm = graycomatrix(img_norm, [1], [0], levels=120, symmetric=True, normed=True)

    return graycoprops(glcm, 'homogeneity')[0, 0]

image = data['samples'].iloc[3]['sample']
print(calculate_glcm_correlation_custom_range(image))
print(get_glcm_contrast(image))
print(get_lbp_smoothness_bin(image))
print(get_lbp_uniform_regularity(image))
print(get_glcm_homogeneity(image))

In [ ]:
import numpy as np

def prepare_frequency_data(image):
    rows, cols = image.shape
    f_transform = np.fft.fft2(image)
    f_shift = np.fft.fftshift(f_transform)

    magnitude = np.abs(f_shift)
    power_spectrum = magnitude**2

    center_y, center_x = rows // 2, cols // 2
    y, x = np.ogrid[:rows, :cols]
    dist_map = np.sqrt((x - center_x)**2 + (y - center_y)**2)
    
    return magnitude, power_spectrum, dist_map

MAX_DIST = np.sqrt(128**2 + 128**2)

def get_high_frequency_energy(power_spectrum, dist_map, threshold_radius=0.8):
    mask = dist_map > (threshold_radius * MAX_DIST)
    return np.sum(power_spectrum[mask])

def get_spectral_centroid(magnitude, dist_map):
    return np.sum(dist_map * magnitude) / (np.sum(magnitude) + 1e-9)

def get_radial_profile_slope(magnitude, dist_map):
    d_flat = dist_map.ravel().astype(int)
    m_flat = magnitude.ravel()

    tbin = np.bincount(d_flat, weights=m_flat)
    nr = np.bincount(d_flat)
    radial_profile = tbin / (nr + 1e-9)

    limit = 128 
    log_dist = np.log(np.arange(1, limit))
    log_power = np.log(radial_profile[1:limit] + 1e-9)
    
    slope, _ = np.polyfit(log_dist, log_power, 1)
    return slope

def get_low_frequency_power(power_spectrum, dist_map, threshold_radius=0.1):
    mask = dist_map < (threshold_radius * MAX_DIST)
    return np.sum(power_spectrum[mask])

image = data['samples'].iloc[3]['sample']
mag, pwr, dists = prepare_frequency_data(image)

features = {
    "hf_energy": get_high_frequency_energy(pwr, dists),
    "centroid":  get_spectral_centroid(mag, dists),
    "slope":     get_radial_profile_slope(mag, dists),
    "lf_power":  get_low_frequency_power(pwr, dists)
}

In [ ]:
def image_stats(img):
    hist, _ = np.histogram(img.flatten(), bins=64, density=True)

    hist = hist[hist > 0]
    ent = -(hist * np.log2(hist)).sum()
    gclm_corr = calculate_glcm_correlation_custom_range(img)
    glcm_contrast = get_glcm_contrast(img)
    mag, pwr, dists = prepare_frequency_data(img)
    return {
        'mean': np.mean(img),
        'std': np.std(img),
        'min': np.min(img),
        'max': np.max(img),
        'kurtosis': kurtosis(img.flatten()),
        'skew': skew(image.flatten()),
        'entropy': ent,
        'gclm_corr': gclm_corr,
        'glcm_contrast': glcm_contrast,
        'glcm_homo': get_glcm_homogeneity(img),
        'lbp_smooth': get_lbp_smoothness_bin(img),
        'lbp_uniform': get_lbp_uniform_regularity(img),
        'high_freq_enenergy': np.log(get_high_frequency_energy(pwr, dists)),
        "centroid":  np.log(get_spectral_centroid(mag, dists)),
        "slope":     get_radial_profile_slope(mag, dists),
        "lf_power":  np.log(get_low_frequency_power(pwr, dists))
    }

image = data['samples'].iloc[0]['sample']
image_stats(image)

## Feature Extraction

### Preprocessing

In [ ]:
def symlog(x):
    return np.sign(x) * np.log10(np.abs(x) + 1)

def preprocess_samples(df):
    df_copy = df.copy()
    df_copy['sample'] = df_copy['sample'].apply(symlog)
    return df_copy

def display_preprocessed_sample(img, filename):
    prep_img = symlog(img)
    print(image_stats(img))
    print(image_stats(prep_img))
    
    # plt.figure(figsize=(4, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap='seismic')
    plt.title('Original Image')
    plt.axis('off')


    plt.subplot(1, 2, 2)
    plt.imshow(prep_img, cmap='seismic')
    plt.title('Preprocessed image')
    plt.axis('off')

    plt.savefig(filename)

image = data['samples'].iloc[0]['sample']
display_preprocessed_sample(image, "preprocessed_image.svg")

### FFT Magnitude

In [ ]:
def compute_spectrum(img, use_log_scale=True):
    f_transform = np.fft.fft2(img)
    f_shift = np.fft.fftshift(f_transform)
    magnitude = np.abs(f_shift)
    
    if use_log_scale:
        magnitude = 20 * np.log(1 + magnitude)

    min_val = np.min(magnitude)
    max_val = np.max(magnitude)
    
    if max_val - min_val == 0:
        return np.zeros_like(magnitude)
        
    normalized = (magnitude - min_val) / (max_val - min_val)
    
    return normalized

def display_spectrum_analysis(img, filename):
    spectrum = compute_spectrum(img, use_log_scale=False)
    log_spectrum = compute_spectrum(img, use_log_scale=True)

    print(f"Image stats: {image_stats(img)}")
    print(f"FFT stats: {image_stats(spectrum)}")
    print(f"Log FFT stats: {image_stats(log_spectrum)}")
    
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='seismic')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(spectrum, cmap='seismic')
    plt.title('Magnitude Spectrum')
    plt.axis('off')

    
    plt.subplot(1, 3, 3)
    plt.imshow(log_spectrum, cmap='seismic')
    plt.title('Log Magnitude Spectrum')
    plt.axis('off')

    plt.suptitle(filename)
    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

image = data['samples'].iloc[0]['sample']
display_spectrum_analysis(image, filename="fft_spectrum.svg")
display_spectrum_analysis(symlog(image), filename="fft_spectrum_prep.svg")

### Radial Profile

In [ ]:
def get_radial_profile(magnitude):
    y, x = np.indices(magnitude.shape)
    center = np.array(magnitude.shape) / 2
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    r = r.astype(int)

    tbin = np.bincount(r.ravel(), weights=magnitude.ravel())
    nr = np.bincount(r.ravel())

    radial_profile = tbin / np.maximum(nr, 1)
    
    return radial_profile

def display_radial_profile(img, filename):
    magnitude = compute_spectrum(img)
    profile = get_radial_profile(magnitude)
    
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='seismic')
    plt.title('Original')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(magnitude, cmap='seismic')
    plt.title('2D Magnitude Spectrum')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.plot(profile, color='blue', linewidth=2)
    plt.title(filename)
    plt.xlabel('Frequency (Radius)')
    plt.ylabel('Average Magnitude')
    plt.grid(True, alpha=0.3)

    plt.suptitle(filename)
    plt.savefig(filename)
    plt.tight_layout()
    plt.show()

image = data['samples'].iloc[0]['sample']
display_radial_profile(image, "radial_profile.svg")
display_radial_profile(symlog(image), "radial_profile_preprocessed.svg")

### Pixel Value Histogram

In [ ]:
def get_pixel_histrogram(img, bins=64):
    return  np.histogram(img.flatten(), bins=bins, density=True)
    
def display_hist(img, bins, filename):
    spectrum = compute_spectrum(img)

    pixel_hist, pixel_bin_edges = get_pixel_histrogram(img, bins)
    fft_hist, fft_bin_edges = get_pixel_histrogram(spectrum, bins)

    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(img, cmap='seismic') 
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(spectrum, cmap='seismic')
    plt.title("FFT Log Magnitude")
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.bar(pixel_bin_edges[:-1], pixel_hist, width=np.diff(pixel_bin_edges), 
            edgecolor='black', align='edge', color='skyblue')
    plt.title("Pixel Intensity Histogram")
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")
    plt.grid(axis='y', alpha=0.3)


    plt.subplot(1, 4, 4)
    plt.bar(fft_bin_edges[:-1], fft_hist, width=np.diff(fft_bin_edges), 
            edgecolor='black', align='edge', color='orange')
    plt.title("FFT Log-Mag Histogram")
    plt.xlabel("Log Magnitude")
    plt.ylabel("Frequency")
    plt.grid(axis='y', alpha=0.3)

    plt.suptitle(filename)
    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

image = data['samples'].iloc[0]['sample']
display_hist(image, 256, "histogram.svg")
display_hist(symlog(image), 256, "histogram_preprocessed.svg")

## Full Feature Set

In [ ]:
def display_feature_set(img, filename):
    print(f"Image Stats: {image_stats(img)}")
    
    img_sym = symlog(img)
    
    spec_orig = compute_spectrum(img)
    spec_sym = compute_spectrum(img_sym)
    
    radial_prof = get_radial_profile(spec_orig)
    
    hist, bin_edges = np.histogram(img_sym.flatten(), bins=128, density=True)

    plt.figure(figsize=(28, 5))
    
    plt.subplot(1, 6, 1)
    plt.imshow(img, cmap='seismic')
    plt.title("1. Original Image (Raw)")
    plt.axis('off')
    
    plt.subplot(1, 6, 2)
    plt.imshow(img_sym, cmap='seismic')
    plt.title("2. Symlog Transformed Image")
    plt.axis('off')

    plt.subplot(1, 6, 3)
    plt.bar(bin_edges[:-1], hist, width=np.diff(bin_edges), edgecolor='black', align='edge', color='skyblue')
    plt.title("3. Pixel Intensity Histogram")
    plt.xlabel("Pixel Value")
    plt.grid(alpha=0.3)

    plt.subplot(1, 6, 4)
    plt.imshow(spec_orig, cmap='seismic')
    plt.title("4. Spectrum (Original)")
    plt.axis('off')

    plt.subplot(1, 6, 5)
    plt.imshow(spec_sym, cmap='seismic')
    plt.title("5. Spectrum (Symlog - Cleaner)")
    plt.axis('off')

    plt.subplot(1, 6, 6)
    plt.plot(radial_prof, color='blue', linewidth=2)
    plt.title("6. Radial Profile (of Spectrum)")
    plt.xlabel("Frequency Radius")
    plt.ylabel("Avg Magnitude")
    plt.grid(alpha=0.3)

    plt.savefig(filename)
    plt.tight_layout()
    plt.show()

for i in range(0, 2):
    image = data['samples'].iloc[i]['sample']
    display_feature_set(image, "full_feature_set.svg")

In [ ]:
def find_and_display_shared_pairs(df_train, df_samples):
    all_ids = pd.concat([df_train['id_noise_1'], df_train['id_noise_2']])
    id_counts = all_ids.value_counts()
    
    reused_ids = id_counts[id_counts >= 2]
    
    if reused_ids.empty:
        print("No shared images found (every image appears in only one pair).")
        return

    shared_id = reused_ids.index[0]
    count = reused_ids.iloc[0]
    
    mask = (df_train['id_noise_1'] == shared_id) | (df_train['id_noise_2'] == shared_id)
    pairs = df_train[mask].head(2)  
    print(f"Found Shared Image ID: {shared_id} (Appears in {count} pairs)")
    
    for i, (idx, row) in enumerate(pairs.iterrows()):
        id1 = row['id_noise_1']
        id2 = row['id_noise_2']
        label = row['label']
        
        print(f"\n{'='*80}")
        print(f"PAIR {i+1}: Label {label} | IDs: {id1} vs {id2}")
        print(f"{'='*80}\n")
        
        img1 = df_samples.loc[id1]['sample']
        img2 = df_samples.loc[id2]['sample']
        
        print(f"--- Image 1 (ID: {id1}) {'[SHARED]' if id1 == shared_id else ''} ---")
        display_feature_set(img1, "img1_set.svg")
        
        print(f"--- Image 2 (ID: {id2}) {'[SHARED]' if id2 == shared_id else ''} ---")
        display_feature_set(img2, "img2_set.svg")

find_and_display_shared_pairs(data['train'], data['samples'])

# Training - XGBoost

### Create feature vector

In [ ]:
def get_feature_vector(img):
    img_sym = symlog(img)
    spec_orig = compute_spectrum(img)
    # spec_sym = compute_spectrum(img_sym)
    
    stats_orig = np.array(list(image_stats(img).values()))
    # stats_sym = np.array(list(image_stats(img_sym).values()))

    # hist, _ = np.histogram(img_sym.flatten(), bins=128, density=True)
    # rad_profile = get_radial_profile(spec_orig)
    
    # return np.concatenate([stats_orig, stats_sym, hist, rad_profile])
    return stats_orig

image = data['samples'].iloc[0]['sample']
print(*get_feature_vector(image))

In [ ]:
def get_combined_features(f1, f2):
    diff = np.abs(f1 - f2)

    eps = 1e-8
    ratio = np.log((np.abs(f1) + eps) / (np.abs(f2) + eps))

    sq_diff = np.square(f1 - f2)

    return np.concatenate([f1, f2, diff, ratio, sq_diff])

def build_dataset(samples_df, pairs_df):
    X_list = []
    y_list = []
    
    for _, row in tqdm(pairs_df.iterrows()):
        id1, id2 = row['id_noise_1'], row['id_noise_2']
        f1 = get_feature_vector(samples_df.loc[id1, 'sample'])
        f2 = get_feature_vector(samples_df.loc[id2, 'sample'])

        combined = get_combined_features(f1, f2)
        
        X_list.append(combined)
        if 'label' in row:
            y_list.append(row['label'])
        
    return np.array(X_list), np.array(y_list)


X_train, y_train = build_dataset(data['samples'], data['train'])
X_val, y_val = build_dataset(data['samples'], data['val'])
X_test, _ = build_dataset(data['samples'], data['test'])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

### Grid Search

In [ ]:
def xgb_grid_search(param_grid, static_params, history_file):
    print(f"Starting Grid Search with {len(ParameterGrid(param_grid))} combinations...")

    for i, params in enumerate(ParameterGrid(param_grid)):
        print(f"\n--- Iteration {i+1} ---")
        print(f"Testing params: {params}")

        current_params = {**static_params, **params}
        model = xgb.XGBClassifier(**current_params)
        model.fit(
            X_train_scaled, y_train,
            eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)],
            verbose=False
        )

        preds = model.predict(X_val_scaled)
        probs = model.predict_proba(X_val_scaled)[:, 1]
    
        val_acc = accuracy_score(y_val, preds)
        val_auc = roc_auc_score(y_val, probs)
    
        print(f"Result -> Accuracy: {val_acc:.4f} | AUC: {val_auc:.4f}")

        log_entry = current_params.copy()
        log_entry.update({
            'val_accuracy': val_acc,
            'val_auc': val_auc,
            'best_iteration': model.best_iteration,
            'best_score': model.best_score,
        })
    
        log_df = pd.DataFrame([log_entry])
    
        if not os.path.isfile(history_file):
            log_df.to_csv(history_file, index=False)
        else:
            log_df.to_csv(history_file, mode='a', header=False, index=False)
    
    print(f"\nGrid search complete. Results saved to {history_file}")

    results = pd.read_csv(history_file)
    best_run = results.loc[results['val_auc'].idxmax()]
    print("\nBest Parameters found by AUC:")
    print(best_run)


param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'min_child_weight': [1, 5],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.7, 1.0]
}

static_params = {
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'n_jobs': -1,
    'eval_metric': 'logloss',
    'random_state': 42
}

history_file = 'grid_search_history.csv'
xgb_grid_search(param_grid, static_params, history_file)

### Model Training

In [ ]:
def train(params):
    print("Training XGBoost...")
    model = xgb.XGBClassifier(**params)

    model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_train_scaled, y_train), (X_val_scaled, y_val)],
        verbose=5
    )

    preds = model.predict(X_val_scaled)
    probs = model.predict_proba(X_val_scaled)[:, 1]
    
    val_acc = accuracy_score(y_val, preds)
    val_auc = roc_auc_score(y_val, probs)
    
    print(f"\nValidation Accuracy: {val_acc:.4f}")
    print(f"Validation AUC: {val_auc:.4f}")
    
    history_file = 'experiment_history.csv'
    
    log_entry = params.copy()
    log_entry.update({
        'val_accuracy': val_acc,
        'val_auc': val_auc,
        'best_iteration': model.best_iteration,
        'best_score': model.best_score,
    })

    log_df = pd.DataFrame([log_entry])

    if not os.path.isfile(history_file):
        log_df.to_csv(history_file, index=False)
    else:
        log_df.to_csv(history_file, mode='a', header=False, index=False)
    
    print(f"Results saved to {history_file}")
    return model



params = {
    'n_estimators': 2000,
    'learning_rate': 0.02,
    'max_depth': 8,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 1,
    'early_stopping_rounds': 200,
    'n_jobs': -1,
    'eval_metric': 'logloss',
    'random_state': 42 
}
model = train(params)


In [ ]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

def train_rf(params):
    print("Training Random Forest...")

    model = RandomForestClassifier(**params)

    model.fit(X_train_scaled, y_train)

    preds = model.predict(X_val_scaled)
    probs = model.predict_proba(X_val_scaled)[:, 1]
    
    val_acc = accuracy_score(y_val, preds)
    val_auc = roc_auc_score(y_val, probs)
    
    print(f"\nValidation Accuracy: {val_acc:.4f}")
    print(f"Validation AUC: {val_auc:.4f}")
    
    history_file = 'experiment_history_rf.csv'

    log_entry = params.copy()
    log_entry.update({
        'val_accuracy': val_acc,
        'val_auc': val_auc,
    })
    
    log_df = pd.DataFrame([log_entry])
    
    if not os.path.isfile(history_file):
        log_df.to_csv(history_file, index=False)
    else:
        log_df.to_csv(history_file, mode='a', header=False, index=False)
    
    print(f"Results saved to {history_file}")
    return model

rf_params = {
    'n_estimators': 100,
    'max_depth': 12,
    'min_samples_split': 5,
    'min_samples_leaf': 5,
    'max_features': 'sqrt',
    'bootstrap': True,
    'n_jobs': -1,
    'random_state': 42 
}

model = train_rf(rf_params)

### Results Analysis

In [ ]:
def validate_extended_metrics(model, X_val, y_val, model_name, filename):
    probs = model.predict_proba(X_val)[:, 1]
    
    mae = mean_absolute_error(y_val, probs)
    mse = mean_squared_error(y_val, probs)
    
    spearman_corr, _ = spearmanr(y_val, probs)
    kendall_corr, _ = kendalltau(y_val, probs)
    
    results = {
        'Model': model_name,
        'MAE': mae,
        'MSE': mse,
        'Spearman_R': spearman_corr,
        'Kendall_Tau': kendall_corr
    }
    
    df_results = pd.DataFrame([results])
    
    print("\n### Validation Performance Report ###")

    display_df = df_results.style.format({
        'MAE': "{:.4f}",
        'MSE': "{:.4f}",
        'Spearman_R': "{:.4f}",
        'Kendall_Tau': "{:.4f}"
    }).hide(axis='index')
    
    from IPython.display import display
    display(display_df)
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    sns.barplot(x=['MAE', 'MSE'], y=[mae, mse], palette="Reds")
    plt.title(f"{model_name}: Error Metrics (Lower is Better)")
    plt.ylabel("Score")
    plt.ylim(0, max(mae, mse) * 1.2)
    for i, v in enumerate([mae, mse]):
        plt.text(i, v, f"{v:.4f}", ha='center', va='bottom', fontweight='bold')

    plt.subplot(1, 2, 2)
    sns.barplot(x=['Spearman', 'Kendall'], y=[spearman_corr, kendall_corr], palette="Greens")
    plt.title(f"{model_name}: Rank Correlations (Higher is Better)")
    plt.ylabel("Correlation Coefficient")
    plt.ylim(0, 1.0) 
    for i, v in enumerate([spearman_corr, kendall_corr]):
        plt.text(i, v, f"{v:.4f}", ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

    return df_results

performance_df = validate_extended_metrics(model, X_val_scaled, y_val, model_name="XGB_GridSearch_Best", filename="xgboost-results.svg")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
import seaborn as sns

def plot_calibration_curve(model, X_val, y_val, n_bins=10):
    probs = model.predict_proba(X_val)[:, 1]
    prob_true, prob_pred = calibration_curve(y_val, probs, n_bins=n_bins, strategy='uniform')

    plt.figure(figsize=(10, 10))

    plt.subplot(2, 1, 1)
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated', color='black')
    plt.plot(prob_pred, prob_true, marker='o', label='XGBoost Model', color='tab:blue')
    
    plt.xlabel("Mean Predicted Probability")
    plt.ylabel("Fraction of Positives")
    plt.title("Calibration Curve (Reliability Diagram)")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.subplot(2, 1, 2)
    sns.histplot(probs, bins=20, kde=True, color='tab:blue', alpha=0.5)
    plt.xlabel("Predicted Probability")
    plt.ylabel("Count")
    plt.title("Distribution of Predicted Probabilities")
    plt.xlim(0, 1)
    
    plt.tight_layout()
    plt.show()

plot_calibration_curve(model, X_val_scaled, y_val)

In [ ]:
model.feature_importances_

In [ ]:
def run_inference(model, test_df, X, output_file='submission.csv'):
    predictions = model.predict(X)
    
    submission_df = pd.DataFrame({
        'id_pair': '(' + test_df['id_noise_1'].astype(str) + ',' + test_df['id_noise_2'].astype(str) + ')',
        'label': predictions
    })

    submission_df.to_csv(output_file, index=False)
    
    print(f"Success! Results saved to {output_file}")
    print("\nPreview:")
    print(submission_df.head())
    return submission_df

run_inference(model, data['test'], X_test_scaled)

# Traininig - Siamese Network

### Dataset

In [ ]:
class NoiseMatchingDataset(Dataset):
    def __init__(self, pairs_df, samples_df, mode='train'):
        self.mode = mode
        self.pairs_df = pairs_df
        self.image_lookup = dict(zip(samples_df.index, samples_df['sample']))

        # if self.mode == 'train':
        #     self.crop = T.RandomCrop(224)
        # else:
        #     self.crop = T.CenterCrop(224)

    def __len__(self):
        return len(self.pairs_df)

    def __getitem__(self, idx):
        row = self.pairs_df.iloc[idx]
        
        id1 = row['id_noise_1']
        id2 = row['id_noise_2']
        if 'label' in row:
            label = float(row['label']) 
        else:
            label = []
        img1_raw = self.image_lookup[id1]
        img2_raw = self.image_lookup[id2]

        img1 = symlog(img1_raw)
        img2 = symlog(img2_raw)

        if self.mode == 'train':
            if random.random() > 0.5:
                img1 = np.fliplr(img1)
            if random.random() > 0.5:
                img2 = np.fliplr(img2)

            if random.random() > 0.5:
                img1 = np.flipud(img1)
            if random.random() > 0.5:
                img2 = np.flipud(img2)

            if random.random() > 0.5:
                img1 = np.rot90(img1)
            if random.random() > 0.5:
                img2 = np.rot90(img2)

        fft1 = compute_spectrum(img1.copy())
        fft2 = compute_spectrum(img2.copy())

        img1_t = torch.tensor(img1.copy(), dtype=torch.float32).unsqueeze(0)
        img2_t = torch.tensor(img2.copy(), dtype=torch.float32).unsqueeze(0)
        fft1_t = torch.tensor(fft1.copy(), dtype=torch.float32).unsqueeze(0)
        fft2_t = torch.tensor(fft2.copy(), dtype=torch.float32).unsqueeze(0)

        # input1 = torch.cat([img1_t, fft1_t], dim=0)
        # input2 = torch.cat([img2_t, fft2_t], dim=0)

        return fft1_t, fft2_t, torch.tensor(label, dtype=torch.float32)

In [ ]:
import torch
import torch.nn as nn

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn = nn.Sequential(
            # Block 1: 256 -> 128
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            # Block 2: 128 -> 64
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            
            # Block 3: 64 -> 32
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            
            # Block 4: 32 -> 16
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),

            # Block 5: 16 -> 8
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2),

            # Block 6: 8 -> 4
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2),
        )
        
        # Flattened size: 512 channels * 4 * 4 pixels = 8192
        self.flatten_dim = 512 * 4 * 4

        # --- Classifier Head ---
        self.fc = nn.Sequential(
            nn.Linear(self.flatten_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward_one(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return x

    def forward(self, input1, input2):
        feat1 = self.forward_one(input1)
        feat2 = self.forward_one(input2)
        
        # L1 Distance
        diff = torch.abs(feat1 - feat2)

        output = self.fc(diff)
        return output

In [ ]:
def run(model, 
        data, 
        epochs=50, 
        learning_rate=0.001, 
        base_batch_size=64, 
        patience=5, 
        save_path="best_noise_model.pth"):

    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        final_batch_size = base_batch_size * torch.cuda.device_count()
    else:
        print("Using single GPU or CPU")
        final_batch_size = base_batch_size
        
    print(f"Final Batch Size: {final_batch_size}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_ds = NoiseMatchingDataset(data['train'], data['samples'], mode='train')
    val_ds = NoiseMatchingDataset(data['val'], data['samples'], mode='val')
    
    train_loader = DataLoader(train_ds, batch_size=final_batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=final_batch_size, shuffle=False, num_workers=4, pin_memory=True)

    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    model = model.to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    best_val_loss = float('inf')
    patience_counter = 0
    train_loss_history = []
    val_loss_history = []
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
        
        for img1, img2, labels in train_loop:
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(img1, img2).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_loop.set_postfix(loss=loss.item(), acc=100 * correct / total)
            
        train_acc = 100 * correct / total
        avg_train_loss = running_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        
        with torch.no_grad():
            for img1, img2, labels in val_loop:
                img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
                
                outputs = model(img1, img2).squeeze()
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                val_loop.set_postfix(val_loss=loss.item())
                
        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)

        train_loss_history.append(avg_train_loss)
        val_loss_history.append(avg_val_loss)

        current_lr = optimizer.param_groups[0]['lr']
        scheduler.step(avg_val_loss)
        
        print(f"Epoch [{epoch+1}] Train: {train_acc:.1f}% ({avg_train_loss:.4f}) | "
              f"Val: {val_acc:.1f}% ({avg_val_loss:.4f}) | LR: {current_lr:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), f"{avg_val_loss}_"+save_path)
            print(f"  >>> New Best Model Saved to {avg_val_loss}_{save_path} (Loss: {best_val_loss:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  >>> Early Stopping triggered after {patience} epochs without improvement.")
                break
    
    print("Training Complete.")

    plt.figure(figsize=(10, 6))
    plt.plot(train_loss_history, label='Training Loss', color='blue')
    plt.plot(val_loss_history, label='Validation Loss', color='orange', linestyle='--')
    plt.xlabel('Epochs')
    plt.ylabel('BCELoss')
    plt.title(f'Loss Curve (Best Val: {best_val_loss:.4f})')
    plt.legend()
    plt.grid(True)
    plt.savefig('loss_graph.png')
    plt.show()



model = SiameseNetwork()
run(
    model=model,
    data=data, 
    epochs=100,
    learning_rate=0.0001,
    base_batch_size=64,
    patience=10,
    save_path="my_model_v1.pth"
)

In [ ]:
def load_model(model, path, input_shape=(1, 256, 256), device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    state_dict = torch.load(path, map_location=device)
    
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith("module."):
            new_state_dict[k[7:]] = v
        else:
            new_state_dict[k] = v
            
    model.load_state_dict(new_state_dict)
    model.to(device)
    model.eval() 
    
    print(f"Model loaded successfully from {path}")
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loaded_model = load_model(SiameseNetwork(), "/kaggle/working/0.4609244247277578_my_model_v1.pth", input_shape=(1, 256, 256), device=device)

In [ ]:
def predict_and_save(model, 
                     samples_df,
                     test_df, 
                     batch_size=64, 
                     save_path="submission.csv"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Predicting on: {device}")
    
    model = load_model(model, "/kaggle/working/0.47040345668792727_my_model_v1.pth")
    
    model = model.to(device)
    model.eval()
    
    test_ds = NoiseMatchingDataset(test_df, samples_df, mode='val') 
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    predictions = []

    print(f"Starting prediction on {len(test_ds)} pairs...")
    
    with torch.no_grad():
        for img1, img2, _ in tqdm(test_loader, desc="Predicting"):
            img1 = img1.to(device)
            img2 = img2.to(device)
            
            outputs = model(img1, img2).squeeze()
            preds = (outputs > 0.5).int().cpu().numpy()
            
            if np.ndim(preds) == 0:
                predictions.append(preds.item())
            else:
                predictions.extend(preds.tolist())

    
    df = pd.DataFrame()
    
    df['id_pair'] = "(" + test_df['id_noise_1'] + ',' + test_df['id_noise_2'] + ")"
    df['label'] = predictions
    
    df.to_csv(save_path, index=False)
    
    print(f"\nSuccess! Predictions saved to: {save_path}")
    print(df.head())
    
    return df

model = SiameseNetwork()

predict_and_save(
    model=model,
    samples_df = data['samples'],
    test_df= data['test'],
    save_path="submission_v1.csv"
)

# Ensemble